[churn dataset](https://bigml.com/user/francisco/gallery/dataset/5163ad540c0b5e5b22000383)

In [118]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
# Carregando pacotes
import numpy as np # Arrays e funções matemáticas.
import pandas as pd # Manipulação de data frames.
import matplotlib.pyplot as plt # Gráficos.
import seaborn as sns # Gráficos.
import scipy.stats as stats # Funções estatísticas.
import random # Random Seed
import warnings

random.seed(42) # Garantir reprodutibilidade.
warnings.filterwarnings("ignore")

In [120]:
%autoreload 2
from source.utils import reduce_mem_usage
from source.utils import resumetable
from source.utils import missing_data
from source.utils import dummy_encode
from source.utils import apply_crosstab

In [125]:
# Carregando os dados
df = pd.read_csv("train_raw.csv")

In [126]:
print("train dataset :{}".format(df.shape))

train dataset :(2666, 21)


In [127]:
df.head(2)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,UT,243,510,355-9360,no,no,0,95.5,92,16.24,...,63,13.91,264.2,118,11.89,6.6,6,1.78,2,False.
1,SC,108,415,399-6233,no,no,0,112.0,105,19.04,...,110,16.46,208.9,93,9.40,4.1,4,1.11,4,True.


In [128]:
df.columns = [c.lower().replace(' ', '_').replace('?', '').replace("'", "") for c in df.columns] 

In [129]:
df.head(2)

,state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,churn
0,UT,243,510,355-9360,no,no,0,95.5,92,16.24,...,63,13.91,264.2,118,11.89,6.6,6,1.78,2,False.
1,SC,108,415,399-6233,no,no,0,112.0,105,19.04,...,110,16.46,208.9,93,9.40,4.1,4,1.11,4,True.


In [130]:
# Procurando por valores nulos.
resumetable(df)

Dataset Shape: (2666, 21)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,state,object,0,51,UT,SC,TX,5.65
1,account_length,int64,0,207,243,108,75,7.27
2,area_code,int64,0,3,510,415,415,1.50
3,phone,object,0,2666,355-9360,399-6233,384-2372,11.38
4,intl_plan,object,0,2,no,no,yes,0.46
5,vmail_plan,object,0,2,no,no,no,0.84
6,vmail_message,int64,0,46,0,0,0,2.19
7,day_mins,float64,0,1505,95.5,112,222.4,10.34
8,day_calls,int64,0,115,92,105,78,6.32
9,day_charge,float64,0,1505,16.24,19.04,37.81,10.34


In [131]:
display(missing_data(df))

,state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,churn
Total,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Percent,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Types,object,int64,int64,object,object,object,int64,float64,int64,float64,...,int64,float64,float64,int64,float64,float64,int64,float64,int64,object


In [132]:
# Quantas transações são churn, percentualmente?
df['churn'].value_counts()/len(df) * 100

False.    85.671418
True.     14.328582
Name: churn, dtype: float64

In [133]:
apply_crosstab(df["state"],df["churn"]).sort_values(['True.'],ascending=False).head(20)

churn,False.,True.
state,,
TX,72.88,27.12
AR,73.17,26.83
NJ,73.58,26.42
SC,74.47,25.53
NV,75.00,25.00
KS,77.36,22.64
CA,77.78,22.22
MD,77.97,22.03
ME,79.17,20.83


In [134]:
apply_crosstab(df["area_code"],df["churn"]).sort_values(['True.'],ascending=False).head(20)

churn,False.,True.
area_code,,
510,83.69,16.31
415,86.19,13.81
408,86.60,13.40
